### Imports

In [1]:
import snscrape.modules.twitter
import pandas as pd
from gensim.models import Doc2Vec
from gensim.utils import simple_preprocess
import pickle
import csv

### Create function that takes in a twitter handle as input and returns that user's probable political bent

In [8]:
def get_bent(twitter_handle):
    
    # import Doc2Vec model
    
    doc_model = Doc2Vec.load('final_doc2vec')
    
    # import SVC model
    
    svc_filename = "svc_final_model.pkl"
    with open(svc_filename, 'rb') as file:
        svc = pickle.load(file)
    
    # scrape all tweets from user
    
    tweets = snscrape.modules.twitter.TwitterUserScraper(username=twitter_handle)
    
    # create temporary file to hold tweets
    
    with open('temp_file7.csv', 'w', encoding='utf-8') as f:
        wr=csv.writer(f, lineterminator = '\n')
        for i in tweets.get_items():
            wr.writerow([i.content])
                    
    # read temporary file in
    
    df = pd.read_csv('temp_file7.csv', header=None)
    
    # transform rows of tweets into single string 
    
    tweets_list = df[0].tolist()
    one_string = ''.join(tweets_list)
    
    # process tweets with gensim
    
    clean_doc = simple_preprocess(one_string)
    
    # infer document embeddings for entirety of user's tweets
    
    vectors = doc_model.infer_vector(clean_doc)
    
    # make prediction in svc model
    
    prediction = svc.predict([vectors])
    proba = svc.predict_proba([vectors])
    most_similar = doc_model.wv.most_similar([vectors])
    # display political bent

    if prediction == 1:
        print(f'Well it seems to me that this twitter user, @{twitter_handle}, is most likely a Democrat.')
    else:
        print(f'Hey now, looks like Mr. or Ms. @{twitter_handle}, just may be a Republican.')
    
    print(prediction)
    print(one_string[:2000])
    print(proba)
    print(vectors)
    
    

In [9]:
get_bent('cnmlhk')


Hey now, looks like Mr. or Ms. @cnmlhk, just may be a Republican.
[0]
@chrisalbon Look at the curve of drug abuse and overdose death in the same time period, you may have a different conclusion if this effort of lockdown worth it.@dylanzhao California is like New York. Everyone hates it, but everyone wants to come.  Only losers and retirees leave.@fuhaixia1015 @monicabi5 犹太人也这么评价犹太人的圈子。@mang047 大城市比如斯德哥尔摩，有可比性。@mang047 瑞典为啥现在没大事了？他们既不戴口罩也不封城。@mang047 @StefanSloft @Fabianduduosaka 那种钱不好拿，看看老郭和他目前的几个敌人的现状就知道了。@mang047 @RFA_Chinese 菜可申请避难，她符合豁免条件。杨的问题是在20年前申请绿卡时候有谎报。@mang047 @David_Liu2018 @FlyInRainS2 @CenturyLy @Rosa20071101 @Kobe94943003 @Xzhangbaiyu @sssssw_fo @Shineling2 @gameove68797884 @IanWu18 @nidaye_34 @holalibin @yanzi121 @majuismail1122 整个事件本来就是大酱的智囊团背后发起的。就是政变。@PeterSchiff @realDonaldTrump Printing money is the solution...@mang047 @shiweijueye0307 @kacmoodyalgeb 目前还有当可上也是福气，总比任精英阶层任意宰割强，至少暂时解气。这样精英阶层要么逃离这个国家，要么认真对待。@mang047 @shiweijueye0307 @kacmoodyalgeb 是这么回事，但这是全球化，美国中产和贫困